# Fine-Tune

In [13]:
import joblib
import numpy as np
import pandas as pd


housing_prepared = joblib.load("data/housing_prepared.pkl")
housing_labels = joblib.load("data/housing_labels.pkl")
housing = joblib.load("data/housing_raw.pkl") 
strat_train_set = joblib.load("data/strat_train_set.pkl")
strat_test_set = joblib.load("data/strat_test_set.pkl")

In [3]:
from sklearn.base import BaseEstimator, TransformerMixin

class DataFrameSelector(BaseEstimator, TransformerMixin):  # 재정의
    def __init__(self, attribute_names):  # 선택할 열 이름 목록을 초기화
        self.attribute_names = attribute_names
    def fit(self, X, y=None):  # fit 메서드 (여기선 학습 필요 없음)
        return self
    def transform(self, X):  # 지정된 열만 NumPy 배열 형태로 추출하여 반환
        return X[self.attribute_names] # .values 제거

In [4]:
from sklearn.base import BaseEstimator, TransformerMixin  # 재정의

# 열 인덱스 설정
rooms_ix, bedrooms_ix, population_ix, household_ix = 3, 4, 5, 6  # 순서대로 total_rooms, total_bedrooms, population, households 열의 인덱스

class CombinedAttributesAdder(BaseEstimator, TransformerMixin):  # 파생 특성 생성용 사용자 정의 변환기 클래스 정의
    def __init__(self, add_bedrooms_per_room=True):  # bedrooms_per_room 특성을 추가할지 여부를 설정하는 하이퍼파라미터
        self.add_bedrooms_per_room = add_bedrooms_per_room
    def fit(self, X, y=None):  # fit 메서드는 아무 작업 없이 self 반환 (필수 메서드)
        return self
    def transform(self, X, y=None):  # transform 메서드에서 새로운 파생 특성들을 계산하여 기존 데이터에 추가
        rooms_per_household = X[:, rooms_ix] / X[:, household_ix]
        population_per_household = X[:, population_ix] / X[:, household_ix]
        if self.add_bedrooms_per_room:
            bedrooms_per_room = X[:, bedrooms_ix] / X[:, rooms_ix]
            return np.c_[X, rooms_per_household, population_per_household, bedrooms_per_room]  # 세 개의 파생 특성 추가
        else:
            return np.c_[X, rooms_per_household, population_per_household]  # 두 개의 파생 특성만 추가

In [7]:
full_pipeline = joblib.load("models/full_pipeline.pkl")

## Grid Search

하이퍼파라미터를 손으로 하나씩 조정하면서 좋은 조합을 찾는 대신 
Scikit-Learn의 `GridSearchCV` 사용
>이 도구에 어떤 하이퍼파라미터들을 실험할지,  
>그리고 각 하이퍼파라미터마다 어떤 값을 시도할지를 알려주기만 하면  
>모든 가능한 하이퍼파라미터 조합을 교차 검증을 통해 평가해 줌

In [8]:
from sklearn.model_selection import GridSearchCV  # 하이퍼파라미터 튜닝을 위한 GridSearchCV 임포트
from sklearn.ensemble import RandomForestRegressor  # 랜덤 포레스트 회귀 모델 임포트

param_grid = [  # 탐색할 하이퍼파라미터 조합 정의
    {'n_estimators': [3, 10, 30], 'max_features': [2, 4, 6, 8]},
    {'bootstrap': [False], 'n_estimators': [3, 10], 'max_features': [2, 3, 4]},
]

forest_reg = RandomForestRegressor()  # 모델 객체 생성

grid_search = GridSearchCV(forest_reg, param_grid, cv=5,  # 그리드 탐색 객체 생성
                           scoring='neg_mean_squared_error',
                           return_train_score=True)

grid_search.fit(housing_prepared, housing_labels)  # 전체 훈련 세트로 학습 및 교차 검증

,estimator,RandomForestRegressor()
,param_grid,"[{'max_features': [2, 4, ...], 'n_estimators': [3, 10, ...]}, {'bootstrap': [False], 'max_features': [2, 3, ...], 'n_estimators': [3, 10]}]"
,scoring,'neg_mean_squared_error'
,n_jobs,None
,refit,True
,cv,5
,verbose,0
,pre_dispatch,'2*n_jobs'
,error_score,nan
,return_train_score,True
,n_estimators,30


어떤 하이퍼파라미터의 값을 전혀 모르겠을 때는,  
10의 거듭제곱(또는 더 세밀하게 탐색하고 싶다면 더 작은 수)을 순서대로 시도해보는 것이 간단한 접근법(이 예제에서 `n_estimators` 하이퍼파라미터)

이 `param_grid`는 Scikit-Learn에게 첫 번째 딕셔너리에 지정된 `n_estimators`와 `max_features` 하이퍼파라미터 값의   
모든 3 × 4 = 12가지 조합을 먼저 평가하라고 지시(이 하이퍼파라미터들 의미는 7장에서)  
그런 다음, 두 번째 딕셔너리에 있는 하이퍼파라미터 조합 2 × 3 = 6가지를 평가하는데,   
이때는 기본값인 `bootstrap=True` 대신 `bootstrap=False`로 설정해서 진행  

In [9]:
grid_search.best_params_  # 가장 좋은 하이퍼파라미터 조합 출력

{'max_features': 6, 'n_estimators': 30}

6과 30은 평가된 값들 중 최대값이므로, 더 높은 값을 사용해 다시 탐색해보는 것이 좋음(점수가 계속해서 개선될 수 있기 때문)

In [10]:
grid_search.best_estimator_  # 가장 성능이 좋은 모델 출력

,n_estimators,30
,criterion,'squared_error'
,max_depth,None
,min_samples_split,2
,min_samples_leaf,1
,min_weight_fraction_leaf,0.0
,max_features,6
,max_leaf_nodes,None
,min_impurity_decrease,0.0
,bootstrap,True
,oob_score,False


GridSearchCV가 refit=True로 초기화되면(기본값),  
교차 검증을 통해 최적의 추정기를 찾은 후 전체 학습 세트로 다시 훈련  
이는 일반적으로 좋은 방법인데, 더 많은 데이터를 제공하면 성능이 향상될 가능성이 높기 때문

In [11]:
cvres = grid_search.cv_results_  # 교차 검증 결과 저장
for mean_score, params in zip(cvres["mean_test_score"], cvres["params"]):
    print(np.sqrt(-mean_score), params)  # RMSE와 하이퍼파라미터 조합 출력

64039.07688566286 {'max_features': 2, 'n_estimators': 3}
55388.37627601818 {'max_features': 2, 'n_estimators': 10}
52767.089614730074 {'max_features': 2, 'n_estimators': 30}
60902.33870322378 {'max_features': 4, 'n_estimators': 3}
52370.38732836383 {'max_features': 4, 'n_estimators': 10}
50229.3273204713 {'max_features': 4, 'n_estimators': 30}
58947.14910200078 {'max_features': 6, 'n_estimators': 3}
52270.089324530134 {'max_features': 6, 'n_estimators': 10}
49939.35577894628 {'max_features': 6, 'n_estimators': 30}
58992.39057956946 {'max_features': 8, 'n_estimators': 3}
52282.13605031126 {'max_features': 8, 'n_estimators': 10}
50398.93673505175 {'max_features': 8, 'n_estimators': 30}
62082.604101650846 {'bootstrap': False, 'max_features': 2, 'n_estimators': 3}
54615.71291616976 {'bootstrap': False, 'max_features': 2, 'n_estimators': 10}
59674.460919855854 {'bootstrap': False, 'max_features': 3, 'n_estimators': 3}
52130.22641702382 {'bootstrap': False, 'max_features': 3, 'n_estimators':

이 예제에서는 `max_features` 하이퍼파라미터를 6으로, `n_estimators` 하이퍼파라미터를 30으로 설정함으로써 최적의 해를 얻음  
이 조합의 RMSE 점수는 50,159로, 이전에 기본 하이퍼파라미터 값을 사용했을 때의 점수인 50,182보다 약간 더 좋음???

데이터 준비 단계 중 일부도 하이퍼파라미터로 다룰 수 있다는 점을 잊지 말아야 함
>예를 들어, 그리드 서치를 통해 확신이 없던 특성(예: `CombinedAttributesAdder` 변환기의 `add_bedrooms_per_room` 하이퍼파라미터)을 추가할지 말지를 자동으로 찾아낼 수 있음  
>이와 유사하게 이상치 처리, 결측 특성 처리, 특성 선택 등에도 최적의 방식을 자동으로 찾아내는 데 사용할 수 있음

In [14]:
pd.DataFrame(grid_search.cv_results_)

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_max_features,param_n_estimators,param_bootstrap,params,split0_test_score,split1_test_score,...,mean_test_score,std_test_score,rank_test_score,split0_train_score,split1_train_score,split2_train_score,split3_train_score,split4_train_score,mean_train_score,std_train_score
0,0.114308,0.012268,0.004874,0.000466,2,3,NaN,"{'max_features': 2, 'n_estimators': 3}",-3.999375e+09,-3.738810e+09,...,-4.101003e+09,2.415028e+08,18,-1.071776e+09,-1.074841e+09,-1.105421e+09,-1.139818e+09,-1.042499e+09,-1.086871e+09,3.313292e+07
1,0.325956,0.003186,0.012269,0.000324,2,10,NaN,"{'max_features': 2, 'n_estimators': 10}",-3.204208e+09,-2.962725e+09,...,-3.067872e+09,9.982504e+07,11,-5.894650e+08,-5.857404e+08,-5.715588e+08,-5.748847e+08,-5.843920e+08,-5.812082e+08,6.811001e+06
2,0.962102,0.021754,0.035483,0.001937,2,30,NaN,"{'max_features': 2, 'n_estimators': 30}",-2.773802e+09,-2.774463e+09,...,-2.784366e+09,9.389723e+07,9,-4.305723e+08,-4.342895e+08,-4.416999e+08,-4.322330e+08,-4.249826e+08,-4.327555e+08,5.437155e+06
3,0.167890,0.003296,0.004514,0.000311,4,3,NaN,"{'max_features': 4, 'n_estimators': 3}",-3.497290e+09,-3.698655e+09,...,-3.709095e+09,1.844681e+08,16,-9.483903e+08,-9.608278e+08,-9.484561e+08,-9.570941e+08,-9.402457e+08,-9.510028e+08,7.247920e+06
4,0.556428,0.009088,0.012824,0.000626,4,10,NaN,"{'max_features': 4, 'n_estimators': 10}",-2.620366e+09,-2.614059e+09,...,-2.742657e+09,1.315535e+08,8,-4.992535e+08,-5.153368e+08,-5.313561e+08,-5.152494e+08,-5.365345e+08,-5.195461e+08,1.323668e+07
5,1.657556,0.011776,0.035378,0.000511,4,30,NaN,"{'max_features': 4, 'n_estimators': 30}",-2.509665e+09,-2.460048e+09,...,-2.522985e+09,1.038404e+08,2,-3.892139e+08,-3.980839e+08,-3.930539e+08,-3.991530e+08,-3.850407e+08,-3.929091e+08,5.316992e+06
6,0.231817,0.007090,0.004686,0.000639,6,3,NaN,"{'max_features': 6, 'n_estimators': 3}",-3.470219e+09,-3.568340e+09,...,-3.474766e+09,6.864963e+07,13,-8.489002e+08,-9.123539e+08,-9.032877e+08,-9.282465e+08,-8.608735e+08,-8.907323e+08,3.057424e+07
7,0.771070,0.028956,0.012414,0.000577,6,10,NaN,"{'max_features': 6, 'n_estimators': 10}",-2.789978e+09,-2.695775e+09,...,-2.732162e+09,6.485892e+07,6,-4.917221e+08,-5.296708e+08,-5.080078e+08,-4.944561e+08,-5.219459e+08,-5.091606e+08,1.487136e+07
8,2.354633,0.027524,0.037451,0.002733,6,30,NaN,"{'max_features': 6, 'n_estimators': 30}",-2.462539e+09,-2.450306e+09,...,-2.493939e+09,9.993839e+07,1,-3.751892e+08,-4.057220e+08,-3.910775e+08,-3.693235e+08,-3.760778e+08,-3.834780e+08,1.323908e+07
9,0.294135,0.007486,0.004379,0.000309,8,3,NaN,"{'max_features': 8, 'n_estimators': 3}",-3.448171e+09,-3.347155e+09,...,-3.480102e+09,1.906937e+08,14,-9.301143e+08,-8.990518e+08,-9.474023e+08,-9.132719e+08,-9.332141e+08,-9.246109e+08,1.676657e+07


## Randomized Search(랜덤 탐색)

그리드 서치 방식은 앞선 예제처럼 비교적 적은 수의 조합을 탐색할 때는 괜찮지만,  
하이퍼파라미터 탐색 공간이 클 경우에는 `RandomizedSearchCV`를 사용하는 것이 더 나은 경우가 많음  
이 클래스는 `GridSearchCV`와 거의 같은 방식으로 사용할 수 있지만, 가능한 모든 조합을 시도하는 대신,   
각 반복에서 하이퍼파라미터마다 임의의 값을 선택해 지정된 수의 무작위 조합을 평가함  

장점  
1. 예를 들어 랜덤 탐색을 1,000회 반복하도록 설정하면, 각 하이퍼파라미터에 대해 1,000개의 서로 다른 값을 탐색하게 됨
(그리드 서치 방식에서는 하이퍼파라미터당 몇 개의 값만 시도됨)  
2. 반복 횟수만 설정함으로써, 하이퍼파라미터 탐색에 얼마만큼의 계산 자원을 할당할지 더 쉽게 제어할 수 있다.

## Ensemble Methods(앙상블 기법)

시스템을 미세 조정하는 또 다른 방법 -> 성능이 좋은 여러 모델을 결합하는 것  
이 집단(또는 “앙상블”)은 개별 모델 중 가장 좋은 것보다도 성능이 더 좋을 때가 많음  
(`Random Forest`가 개별 `Decision Tree`보다 더 좋은 성능을 보이는 것처럼)  
특히 개별 모델들이 서로 매우 다른 유형의 오류를 범할 때 효과가 큼

이 주제는 7장에서 더 자세히 다뤄짐


## Analyze the Best Models and Their Errors

최고의 모델들을 살펴보면 문제에 대한 좋은 통찰을 얻는 경우가 많음  
예를 들어, `RandomForestRegressor`는 정확한 예측을 위해 각 특성이 얼마나 중요한지를 나타내는 상대적 중요도를 제공할 수 있음

In [15]:
feature_importances = grid_search.best_estimator_.feature_importances_  # 특성 중요도 추출

In [16]:
feature_importances

array([7.67705565e-02, 6.55763925e-02, 4.18425143e-02, 1.92173410e-02,
       1.65130498e-02, 1.75566350e-02, 1.67292652e-02, 3.67499111e-01,
       4.62140170e-02, 1.06426920e-01, 7.38973050e-02, 1.49742104e-02,
       1.29057501e-01, 1.70385692e-04, 3.49261289e-03, 4.06218288e-03])

이 중요도 점수를 해당 속성 이름 옆에 표시

In [31]:
full_pipeline = joblib.load("full_pipeline.pkl")

# 이후 사용
full_pipeline.fit(housing)  # fit을 다시 할 수도 있고, 이미 fit된 경우 생략 가능
cat_encoder = full_pipeline.named_transformers_["cat"]
cat_one_hot_attribs = list(cat_encoder.get_feature_names_out(cat_attribs))


In [33]:
extra_attribs = ["rooms_per_hhold", "pop_per_hhold", "bedrooms_per_room"]
#cat_encoder = cat_pipeline.named_steps["cat_encoder"] # old solution
cat_encoder = full_pipeline.named_transformers_["cat"]
cat_one_hot_attribs = list(cat_encoder.categories_[0])
attributes = num_attribs + extra_attribs + cat_one_hot_attribs
sorted(zip(feature_importances, attributes), reverse=True) # 특성 중요도 내림차순 정렬

[(np.float64(0.3674991105070475), 'median_income'),
 (np.float64(0.12905750127983318), 'INLAND'),
 (np.float64(0.10642692009153891), 'pop_per_hhold'),
 (np.float64(0.0767705564933417), 'longitude'),
 (np.float64(0.07389730498606496), 'bedrooms_per_room'),
 (np.float64(0.06557639250250427), 'latitude'),
 (np.float64(0.046214016991503425), 'rooms_per_hhold'),
 (np.float64(0.04184251429060201), 'housing_median_age'),
 (np.float64(0.01921734100599543), 'total_rooms'),
 (np.float64(0.01755663504519956), 'population'),
 (np.float64(0.01672926515058307), 'households'),
 (np.float64(0.016513049819697354), 'total_bedrooms'),
 (np.float64(0.014974210372817204), '<1H OCEAN'),
 (np.float64(0.00406218288025789), 'NEAR OCEAN'),
 (np.float64(0.00349261289098004), 'NEAR BAY'),
 (np.float64(0.00017038569203360574), 'ISLAND')]

이 정보를 바탕으로 덜 유용한 특성들을 제거해볼 수도 있음  
(예: `ocean_proximity` 카테고리 중 실제로 유용한 것은 하나뿐인 것 같으므로 나머지는 제거해볼 수 있음)  
또한 시스템이 어떤 구체적인 오류를 범하는지도 살펴보고, 왜 그런 오류가 발생하는지,  
그리고 이를 어떻게 해결할 수 있을지 고민해봐야 함(추가적인 특성을 더하거나, 정보가 없는 특성을 제거하거나, 이상치를 정리하는 등)

## Evaluate Your System on the Test Set

최종 모델을 테스트 세트에서 평가
- 테스트 세트에서 입력값과 레이블을 가져오고, 
- 전체 파이프라인에서 transform()을 호출해 데이터를 변환한 다음(테스트 세트를 학습시키지 않기 위해 fit_transform()이 아니라 transform()을 호출해야), 
- 테스트 세트에서 최종 모델을 평가

In [34]:
from sklearn.metrics import mean_squared_error  # RMSE 계산을 위한 모듈 임포트

final_model = grid_search.best_estimator_  # 최종 모델 지정

X_test = strat_test_set.drop("median_house_value", axis=1)  # 테스트 입력 특성
y_test = strat_test_set["median_house_value"].copy()  # 테스트 타깃 레이블

X_test_prepared = full_pipeline.transform(X_test)  # 전처리 파이프라인 적용
final_predictions = final_model.predict(X_test_prepared)  # 예측 수행

final_mse = mean_squared_error(y_test, final_predictions)  # MSE 계산
final_rmse = np.sqrt(final_mse)  # RMSE 계산
print(final_rmse)  # RMSE 출력

47515.680761036194


어떤 경우에는 일반화 오차에 대한 이러한 점 추정값만으로는 실제로 모델을 배포할지 확신하기에 충분하지 않을 수 있음  
예를 들어, 현재 운영 중인 모델보다 단지 0.1%만 더 나은 경우라면, 이 추정값이 얼마나 정확한지를 알고 싶을 수도 있음  

이를 위해 `scipy.stats.t.interval()`을 사용해 일반화 오차에 대한 95% 신뢰 구간을 계산할 수 있음

In [35]:
from scipy import stats

confidence = 0.95
squared_errors = (final_predictions - y_test) ** 2
np.sqrt(stats.t.interval(confidence, len(squared_errors) - 1,
                         loc=squared_errors.mean(),
                         scale=stats.sem(squared_errors)))

array([45533.36170491, 49418.5472087 ])

수동으로 구간 계산

In [36]:
m = len(squared_errors)
mean = squared_errors.mean()
tscore = stats.t.ppf((1 + confidence) / 2, df=m - 1)
tmargin = tscore * squared_errors.std(ddof=1) / np.sqrt(m)
np.sqrt(mean - tmargin), np.sqrt(mean + tmargin)

(np.float64(45533.36170491447), np.float64(49418.54720870333))

t-점수 대신 z-점수 사용

In [37]:
zscore = stats.norm.ppf((1 + confidence) / 2)
zmargin = zscore * squared_errors.std(ddof=1) / np.sqrt(m)
np.sqrt(mean - zmargin), np.sqrt(mean + zmargin)

(np.float64(45533.95572691289), np.float64(49417.99988090318))

하이퍼파라미터 튜닝을 많이 했다면, 일반적으로 교차 검증을 통해 측정한 성능보다 실제 성능이 약간 낮게 나올 수 있음
>이는 시스템이 검증 데이터에 잘 작동하도록 지나치게 최적화된 결과이며,   
미지의 데이터셋에서는 동일한 성능을 내지 못할 가능성이 크기 때문

이 예제에서는 해당하지 않지만, 이런 일이 발생할 경우 테스트 세트에서 수치를 좋게 만들기 위해 하이퍼파라미터를 조정하고 싶은 유혹을 반드시 참아야 함
-> 새로운 데이터에 일반화되지 않을 가능성이 높기 때문

이제 프로젝트 사전 출시 단계
 - 해결책을 발표하고,
 - 무엇을 배웠는지,
 - 무엇이 잘 작동했고 무엇이 그렇지 않았는지,
 - 어떤 가정을 했는지,
 - 시스템의 한계는 무엇인지 등을 강조

모든 것을 문서화하고, 명확한 시각 자료와 기억하기 쉬운 표현(예: “중간 소득이 주택 가격을 예측하는 가장 중요한 요인이다”)을 담은   
보기 좋은 발표 자료를 만들어야 함  

이 캘리포니아 주택 예제에서 시스템의 최종 성능은 전문가들의 가격 추정보다 나은 수준은 아니며,  
전문가들의 추정치는 대개 20% 정도 오차가 있었음  

하지만 이 시스템이 전문가들의 시간을 절약해 주고,   
그들이 더 흥미롭고 생산적인 업무에 집중할 수 있도록 한다면 출시하는 것도 여전히 좋은 선택일 수 있음  

In [22]:
joblib.dump(grid_search, "models/grid_search.pkl")

['models/grid_search.pkl']